<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Нужно защитить данные клиентов страховой компании «Хоть потоп». Разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуем корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [19]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [20]:
data = pd.read_csv('/datasets/insurance.csv')

In [21]:
display(data.head())
print()
data.info()
print('Количество дубликатов:', data.duplicated().sum())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
Количество дубликатов: 153


Датасет состоит из 5000 строк и 5 столбцов. 

Пропуски отсутствуют, но присутствуют полные дубликаты - избавимся от них.

Тип данных приведем везде к целочисленному.

In [22]:
data = data.drop_duplicates()
data[['Возраст', 'Зарплата']] = data[['Возраст', 'Зарплата']].astype(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4847 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Пол                4847 non-null   int64
 1   Возраст            4847 non-null   int64
 2   Зарплата           4847 non-null   int64
 3   Члены семьи        4847 non-null   int64
 4   Страховые выплаты  4847 non-null   int64
dtypes: int64(5)
memory usage: 227.2 KB


In [23]:
data.corr()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
Пол,1.000000,0.001953,0.015456,-0.007315,0.011565
Возраст,0.001953,1.000000,-0.017386,-0.009064,0.654964
Зарплата,0.015456,-0.017386,1.000000,-0.031687,-0.013123
Члены семьи,-0.007315,-0.009064,-0.031687,1.000000,-0.039303
Страховые выплаты,0.011565,0.654964,-0.013123,-0.039303,1.000000


## Умножение матриц

**Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?**  

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Не изменится

**Обоснование:** 

$X' = XP$, где Р - обратимая матрица, на которую умножается матрица Х.

Рассчитываем новый вектор весов:

$w' = ((XP)^T XP)^{-1} (XP)^T y = (P^T X^T XP)^{-1} P^T X^T y = P^{-1}(X^T X)^{-1}(P^T)^{-1} P^T X^T y = P^{-1}w$, где $(P^T)^{-1} P^T = E$

Тогда:

$a' = X'w' = XPP^{-1} w = XEw = Xw = a$

Предсказания модели не изменились, соответственно качество осталось прежним.

## Алгоритм преобразования

**Алгоритм**

- Создадим случайную матрицу Р с размерностью, равной количеству признаков.

- Проверим матрицы Р на обратимость. Если матрица вдруг окажется необратимой, создадим новую. 

- Умножим матрицу Р на матрицу признаков Х и получим матрицу преобразованных признаков Х'.

**Обоснование**

При умножении признаков на обратимую матрицу качество линейной регрессии не меняется, а значит алгоритм поможет защитить персональную информацию клиентов, но не повлияет на работу модели.

## Проверка алгоритма
Разобьем датасет на целевой признак и остальные.

In [24]:
features = data.drop(['Страховые выплаты'], axis=1)
target = data['Страховые выплаты']

print('Признаки:', features.shape)
print('Целевой признак:', target.shape)

Признаки: (4847, 4)
Целевой признак: (4847,)


Создадим случайную матрицу и проверим ее на обратимость.

In [26]:
def get_rand_matrix():
    det = 0
    while det == 0:
        matrix = np.random.normal(size=(data.shape[1], data.shape[1]))
        det = np.linalg.det(matrix)
    return matrix

matrix

array([[-0.86112647, -0.74285491,  0.88799068,  0.27984951],
       [-0.74785649,  0.92312392, -0.05357419,  1.56107771],
       [-0.45096192, -0.62178957,  1.40967134,  0.32745623],
       [-0.85487722,  1.00814378,  1.26778959, -1.19102091]])

Умножим признаки на обратимую матрицу.

In [27]:
new_matrix = features.values @ matrix
new_matrix

array([[-22400.08925388, -30802.64928153,  69919.65788344,
         16304.92194383],
       [-17171.80916077, -23584.53179994,  53566.31443552,
         12513.95523339],
       [ -9491.88811677, -13030.81036751,  29601.54456514,
          6921.85205025],
       ...,
       [-15304.27590533, -21058.18764276,  47789.32264503,
         11129.60565543],
       [-14766.33332037, -20309.92862256,  46099.76566455,
         10738.86916537],
       [-18331.70985727, -25218.54376627,  57233.31225506,
         13337.52187789]])

**Обучение без преобразования**

Разделим данные на обучающую и тестовую выборки, обучим модель линейной регрессии и посчитаем метрику r2.

In [28]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [29]:
model_1 = LinearRegression()
model_1.fit(features_train, target_train)
predictions_1 = model_1.predict(features_test)
r2_1 = r2_score(target_test, predictions_1)

print('R2:', r2_1)

R2: 0.42307727615837565


**Обучение с преобразованием**

Так же разделим данные, обучим модель и посчитаем r2.

In [30]:
enc_features_train, enc_features_test, enc_target_train, enc_target_test = train_test_split(
    new_matrix, target, test_size=0.25, random_state=12345)

In [31]:
model_2 = LinearRegression()
model_2.fit(enc_features_train, enc_target_train)
predictions_2 = model_2.predict(enc_features_test)
r2_2 = r2_score(enc_target_test, predictions_2)

print('R2:', r2_2)

R2: 0.4230772761584465


Качество почти не изменилось.

# Вывод

Для защиты данных клиентов был предложен метод преобразования данных путем умножения признаков на обратимую матрицу, не теряя при этом качество линейной регрессии.